In [3]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Markdown, display, HTML
from collections import defaultdict

import tensorflow as tf
from tensorflow import keras
from keras import layers

# Fix the dying kernel problem (only a problem in some installations - you can remove it, if it works without it)
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

# Load the dataset for recommenders

In [4]:
data_path = os.path.join("data", "hotel_data")

data_preprocessed = pd.read_csv(os.path.join(data_path, "hotel_data_preprocessed.csv"), index_col=0)

interactions_df = pd.read_csv(os.path.join(data_path, "hotel_data_interactions_df.csv"), index_col=0)

base_item_features = ['term', 'length_of_stay_bucket', 'rate_plan', 'room_segment', 'n_people_bucket', 'weekend_stay']

column_values_dict = {
    'term': ['WinterVacation', 'Easter', 'OffSeason', 'HighSeason', 'LowSeason', 'MayLongWeekend', 'NewYear', 'Christmas'],
    'length_of_stay_bucket': ['[0-1]', '[2-3]', '[4-7]', '[8-inf]'],
    'rate_plan': ['Standard', 'Nonref'],
    'room_segment': ['[0-160]', '[160-260]', '[260-360]', '[360-500]', '[500-900]'],
    'n_people_bucket': ['[1-1]', '[2-2]', '[3-4]', '[5-inf]'],
    'weekend_stay': ['True', 'False']
}

interactions_df['term'] = pd.Categorical(
    interactions_df['term'], categories=column_values_dict['term'])
interactions_df['length_of_stay_bucket'] = pd.Categorical(
    interactions_df['length_of_stay_bucket'], categories=column_values_dict['length_of_stay_bucket'])
interactions_df['rate_plan'] = pd.Categorical(
    interactions_df['rate_plan'], categories=column_values_dict['rate_plan'])
interactions_df['room_segment'] = pd.Categorical(
    interactions_df['room_segment'], categories=column_values_dict['room_segment'])
interactions_df['n_people_bucket'] = pd.Categorical(
    interactions_df['n_people_bucket'], categories=column_values_dict['n_people_bucket'])
interactions_df['weekend_stay'] = interactions_df['weekend_stay'].astype('str')
interactions_df['weekend_stay'] = pd.Categorical(
    interactions_df['weekend_stay'], categories=column_values_dict['weekend_stay'])

# Data before bucketing
data_preprocessed = data_preprocessed.loc[:, ['user_id', 'item_id', 'night_price', 'n_people', 'length_of_stay']]

display(interactions_df.head(15))
display(data_preprocessed.head(15))

,user_id,item_id,term,length_of_stay_bucket,rate_plan,room_segment,n_people_bucket,weekend_stay
0,1,0,WinterVacation,[2-3],Standard,[260-360],[5-inf],True
1,2,1,WinterVacation,[2-3],Standard,[160-260],[3-4],True
2,3,2,WinterVacation,[2-3],Standard,[160-260],[2-2],False
3,4,3,WinterVacation,[4-7],Standard,[0-160],[3-4],True
4,5,4,WinterVacation,[4-7],Standard,[0-160],[2-2],True
5,6,5,Easter,[4-7],Standard,[160-260],[5-inf],True
6,7,6,OffSeason,[2-3],Standard,[160-260],[5-inf],True
7,8,7,HighSeason,[2-3],Standard,[160-260],[1-1],True
8,9,8,HighSeason,[2-3],Standard,[0-160],[1-1],True
9,8,7,HighSeason,[2-3],Standard,[160-260],[1-1],True


,user_id,item_id,night_price,n_people,length_of_stay
0,1,0,330.76,5,3
1,2,1,231.13,4,3
2,3,2,183.40,2,2
3,4,3,152.09,4,7
4,5,4,142.60,2,5
5,6,5,200.00,5,4
6,7,6,201.00,5,2
7,8,7,220.00,1,3
8,9,8,160.00,1,2
9,8,7,240.00,1,3


# Define user features based on reservations

The content-based recommenders will be forecasting the probability of interaction between user and item based on user features vector and item features vector:

<center>
$$
    r_{u, i} = f(user\_features, item\_features)
$$
</center>

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Design numerical user features based on user reservations. Code the following method which for a given interactions DataFrame (it will be used in the fit method of the recommender) returns a DataFrame with user_id and user features as well as a list with names of user features (this will be important to select the right columns for an ML algorithm). Remember to name the columns differently than item features which you will create in the next task. Validate your features on users with several interactions (sample user ids are already given below).

Ideas for user features:
- Find the vector of most popular feature values from all user reservations and encode every feature with one-hot encoding.
- For every reservation feature calculate the probability distribution of its values among all user's reservations.
- For numerical buckets (length_of_stay, room_segment, n_people) you can calculate the average value for every user from their reservations (you will have to map the buckets back to numerical values before averaging them).

Remember that you will have to select the best features (with the highest explanatory power). Using all above features at once would make the number of variables too large for this dataset and would also introduce too much correlations between features.

You can also prepare several versions of the prepare_users_df method and test which works best in your recommender.

# User features #2

In [5]:
from sklearn.preprocessing import StandardScaler

def prepare_users_df(interactions_df):
    # Global variable contain data before bucketing
    global data_preprocessed
    
    # Make copy of interactions_df
    users_df = interactions_df.copy()
    
    # Get dummies
    users_df = pd.get_dummies(users_df, columns=[
        'term',
        #'length_of_stay_bucket',
        'rate_plan',
        #'room_segment',
        #'n_people_bucket',
        'weekend_stay'
    ])
    
    # Drop duplicates and merge with non-bucket data on user_id and item_id columns
    users_df = users_df.drop_duplicates().merge(data_preprocessed, on=['user_id', 'item_id'])
    
    # Drop unnecessary bucket columns
    users_df = users_df.drop(columns=['item_id', 'length_of_stay_bucket', 'room_segment', 'n_people_bucket'])
    
    # Groupby data by user_id
    users_df = users_df.groupby('user_id').agg(
        {
            'term_WinterVacation': 'sum',
            'term_Easter': 'sum',
            'term_OffSeason': 'sum',
            'term_HighSeason': 'sum',
            'term_LowSeason': 'sum',
            'term_MayLongWeekend': 'sum',
            'term_NewYear': 'sum',
            'term_Christmas': 'sum',
            'rate_plan_Standard': 'sum',
            'rate_plan_Nonref': 'sum',
            'weekend_stay_True': 'sum',
            'weekend_stay_False': 'sum',
            'night_price': 'mean',
            'n_people': 'mean',
            'length_of_stay': 'mean',
        }
    )
    
    # Dividing the numerical (bucket) data in order to obtain the probability distribution of features for a given user
    users_df[list(set(users_df.columns.tolist()) - set(['night_price', 'n_people', 'length_of_stay']))] = users_df[list(set(users_df.columns.tolist()) - set(['night_price', 'n_people', 'length_of_stay']))].div(users_df[list(set(users_df.columns.tolist()) - set(['night_price', 'n_people', 'length_of_stay']))].sum(axis=1) / 3, axis=0)
    
    # Copy of DataFrame
    users_copy = users_df.copy()
    col_names = ['night_price', 'n_people', 'length_of_stay']
    
    # Scale data
    scaler = StandardScaler()
    users_copy = users_copy[col_names]
    scaler.fit(users_copy.values)
    scaled_data = scaler.transform(users_copy)
    
    users_df[col_names] = scaled_data
    
    # Add prefix to columns
    users_df = users_df.add_prefix('user_')
    
    # Reset index
    users_df = users_df.reset_index()
    
    # Get list of user features - dot product features
    user_features_dot_prod = users_df.columns.tolist()[1:-3]
    
    # Get list of user features - other features
    user_features = users_df.columns.tolist()[-3:]
    
    return users_df, user_features_dot_prod, user_features
    

users_df, user_features_dot_prod, user_features = prepare_users_df(interactions_df)

display(user_features_dot_prod)

display(user_features)

display(users_df.loc[users_df['user_id'].isin([706, 1736, 7779, 96, 1, 50, 115])].head(15))

['user_term_WinterVacation',
 'user_term_Easter',
 'user_term_OffSeason',
 'user_term_HighSeason',
 'user_term_LowSeason',
 'user_term_MayLongWeekend',
 'user_term_NewYear',
 'user_term_Christmas',
 'user_rate_plan_Standard',
 'user_rate_plan_Nonref',
 'user_weekend_stay_True',
 'user_weekend_stay_False']

['user_night_price', 'user_n_people', 'user_length_of_stay']

,user_id,user_term_WinterVacation,user_term_Easter,user_term_OffSeason,user_term_HighSeason,user_term_LowSeason,user_term_MayLongWeekend,user_term_NewYear,user_term_Christmas,user_rate_plan_Standard,user_rate_plan_Nonref,user_weekend_stay_True,user_weekend_stay_False,user_night_price,user_n_people,user_length_of_stay
0,1,0.090909,0.0,0.681818,0.090909,0.136364,0.000000,0.000000,0.0,0.500000,0.500000,0.772727,0.227273,-0.342375,-0.494209,0.309570
40,50,0.043478,0.0,0.434783,0.304348,0.217391,0.000000,0.000000,0.0,0.260870,0.739130,0.782609,0.217391,0.520292,0.431490,-0.142167
84,96,0.090909,0.0,0.681818,0.136364,0.045455,0.045455,0.000000,0.0,0.272727,0.727273,0.727273,0.272727,-0.353219,-0.123469,-0.393917
102,115,0.000000,0.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,1.000000,0.000000,0.000000,1.000000,-0.293730,0.114864,0.038998
371,706,0.095238,0.0,0.511905,0.190476,0.142857,0.047619,0.011905,0.0,0.976190,0.023810,0.583333,0.416667,0.064042,0.128735,0.216158
1383,1736,0.034483,0.0,0.482759,0.206897,0.275862,0.000000,0.000000,0.0,0.172414,0.827586,0.448276,0.551724,-0.061058,-0.819292,-0.166263
7301,7779,0.000000,0.0,0.500000,0.000000,0.500000,0.000000,0.000000,0.0,1.000000,0.000000,0.750000,0.250000,0.153318,0.551807,1.080700


# Prepare numerical item features

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Code the prepare_items_df method which will be used in the recommender fit and recommend methods to map items to numerical features. This method should take the interactions_df DataFrame as input and return a DataFrame containing one record per item_id with item_id column and numerical item feature columns.

You can try turning all item features into one-hot representations. You can use the get_dummies method from pandas. It will return the same columns on any dataset of interactions because the categorical variables with all possible values have been defined in the second cell in this notebook.

You are welcome to design your own numerical item features, for instance based on numerical min and max values in buckets used as features.

## Item features #2

In [6]:
from sklearn.preprocessing import StandardScaler

def prepare_items_df(interactions_df):
    global data_preprocessed
    
    # Create copy of DataFrame
    items_df = interactions_df.copy()
    
    # Make copy of data_preprocessed
    data_prep = data_preprocessed.copy()
    
    # Delete user_id column
    data_prep = data_prep.drop(columns=['user_id'])
    
    # Get dummies
    items_df = pd.get_dummies(items_df, columns=[
        'term',
        #'length_of_stay_bucket',
        'rate_plan',
        #'room_segment',
        #'n_people_bucket',
        'weekend_stay'
    ])
    
    # Merge data with non-bucket data on item_id column
    items_df = items_df.merge(data_prep, on=['item_id'])
    
    # Drop unnecessary bucket columns
    items_df = items_df.drop(columns=['length_of_stay_bucket', 'room_segment', 'n_people_bucket'])
    
    # Drop user column if exists
    if 'user_id' in items_df.columns:
        items_df = items_df.drop(columns=['user_id'])
    
    # Groupby data by item_id
    items_df = items_df.groupby('item_id').agg(
        {
            'term_WinterVacation': 'sum',
            'term_Easter': 'sum',
            'term_OffSeason': 'sum',
            'term_HighSeason': 'sum',
            'term_LowSeason': 'sum',
            'term_MayLongWeekend': 'sum',
            'term_NewYear': 'sum',
            'term_Christmas': 'sum',
            'rate_plan_Standard': 'sum',
            'rate_plan_Nonref': 'sum',
            'weekend_stay_True': 'sum',
            'weekend_stay_False': 'sum',
            'night_price': 'mean',
            'n_people': 'mean',
            'length_of_stay': 'mean',
        }
    )
    # Dividing the numerical data in order to obtain the probability distribution of features for a given item
    items_df[list(set(items_df.columns.tolist()) - set(['night_price', 'n_people', 'length_of_stay']))] = items_df[list(set(items_df.columns.tolist()) - set(['night_price', 'n_people', 'length_of_stay']))].div(items_df[list(set(items_df.columns.tolist()) - set(['night_price', 'n_people', 'length_of_stay']))].sum(axis=1) / 3, axis=0)
    
    # Copy of DataFrame
    items_copy = items_df.copy()
    col_names = ['night_price', 'n_people', 'length_of_stay']
    
    # Scale data
    scaler = StandardScaler()
    items_copy = items_copy[col_names]
    scaler.fit(items_copy.values)
    scaled_data = scaler.transform(items_copy)
    
    items_df[col_names] = scaled_data
    
    # Reset index
    items_df = items_df.reset_index()
    
    # Get list of item features - dot product features
    item_features_dot_prod = items_df.columns.tolist()[1:-3]
    
    # Get list of item features - other features
    item_features = items_df.columns.tolist()[-3:]
    
    return items_df, item_features_dot_prod, item_features

items_df, item_features_dot_prod, item_features = prepare_items_df(interactions_df)

display(item_features_dot_prod)

display(items_df.loc[items_df['item_id'].isin([0, 1, 2, 3, 4, 5, 6])].head(15))

['term_WinterVacation',
 'term_Easter',
 'term_OffSeason',
 'term_HighSeason',
 'term_LowSeason',
 'term_MayLongWeekend',
 'term_NewYear',
 'term_Christmas',
 'rate_plan_Standard',
 'rate_plan_Nonref',
 'weekend_stay_True',
 'weekend_stay_False']

,item_id,term_WinterVacation,term_Easter,term_OffSeason,term_HighSeason,term_LowSeason,term_MayLongWeekend,term_NewYear,term_Christmas,rate_plan_Standard,rate_plan_Nonref,weekend_stay_True,weekend_stay_False,night_price,n_people,length_of_stay
0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.014099,1.276936,-0.180886
1,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,-0.421093,0.145089,-0.357925
2,2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,-0.435494,-0.775480,-0.329585
3,3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,-0.731288,0.051900,0.772717
4,4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,-0.744852,-0.775480,0.697217
5,5,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,-0.322404,1.020384,0.473387
6,6,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,-0.388477,1.437077,-0.335495


# Content-based recommender

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Code the content-based recommender. User features should be calculated within the fit method based on available training data and should be saved in the object as self.users_df for later use in the recommend method. Item features should be calculated both in the fit method (from interactions_df) and in the recommend method (from items_df - the items to be evaluated).

In the fit method you have to randomly generate non-existing interactions and add them to the training data for the regressor. You should add the target variable to interactions - equal to 1 for real ("positive") interactions and equal to 0 for those newly added "negative" interactions. Generate several negative interactions per every positive interaction (n_neg_per_pos). Treat the proportion as a tunable parameter of the model.

Remember to keep control over randomness - in the init method add seed as a parameter and initialize the random seed generator with that seed:

```python
self.seed = seed
self.rng = np.random.RandomState(seed=seed)
```

Below the base content-based recommender class there are several classes which inherit from the base class and use different ML models:
  - LinearRegressionCBUIRecommender - based on linear regression,
  - SVRCBUIRecommender - based on Support Vector Regressor (if you want to test it, sample the data in the fit method, as the training can take many hours on the entire dataset of interactions),
  - RandomForestCBUIRecommender - based on Random Forest,
  - XGBoostCBUIRecommender - based on XGBoost.
  
There is no need to change anything in those inheriting classes, although you can experiment with other tunable parameters of the underlying models.

You are encouraged to experiment with:
  - Other numerical user and item features (but always train and evaluate the model on buckets defined in the first notebook).
  - Other ML models, e.g. Huber regression, Lasso regression, Ridge regression, LARS regression, Linear SVR, Decision Tree, Naive Bayes, LightGBM, Neural Networks or any model of your choice.

# Generate negative interactions

In [7]:
def generateNegativeInteractions(interactions_df, seed=6789, n_neg_per_pos=5):
    
    # Set random seed
    rng = np.random.RandomState(seed=seed)
    
    # Get number of users and items
    n_users = np.max(interactions_df['user_id']) + 1
    n_items = np.max(interactions_df['item_id']) + 1
    
    # Generate interaction matrix
    r = np.zeros(shape=(n_users, n_items))
    for idx, interaction in interactions_df.iterrows():
        r[int(interaction['user_id'])][int(interaction['item_id'])] = 1
    
    # Generate negative interactions
    
    negative_interactions = []
    
    i = 0
    while i < n_neg_per_pos * len(interactions_df):
        sample_size = 1000
        user_ids = rng.choice(np.arange(n_users), size=sample_size)
        item_ids = rng.choice(np.arange(n_items), size=sample_size)

        j = 0
        while j < sample_size and i < n_neg_per_pos * len(interactions_df):
            if r[user_ids[j]][item_ids[j]] == 0:
                negative_interactions.append([user_ids[j], item_ids[j], 0])
                i += 1
            j += 1
    
    return negative_interactions

negative_interactions = generateNegativeInteractions(interactions_df)
display(negative_interactions[:50])



[[10077, 10, 0],
 [2847, 810, 0],
 [14316, 64, 0],
 [9409, 234, 0],
 [14080, 839, 0],
 [966, 505, 0],
 [13510, 128, 0],
 [6813, 788, 0],
 [802, 887, 0],
 [10612, 404, 0],
 [1351, 243, 0],
 [4846, 1059, 0],
 [1339, 930, 0],
 [6464, 349, 0],
 [8736, 631, 0],
 [4277, 554, 0],
 [254, 125, 0],
 [3305, 38, 0],
 [10420, 196, 0],
 [3500, 493, 0],
 [8539, 1041, 0],
 [13197, 231, 0],
 [7780, 1006, 0],
 [8452, 670, 0],
 [2033, 87, 0],
 [12778, 8, 0],
 [2661, 718, 0],
 [1907, 1050, 0],
 [13433, 613, 0],
 [13392, 581, 0],
 [5554, 883, 0],
 [6213, 811, 0],
 [1732, 259, 0],
 [13390, 362, 0],
 [1179, 415, 0],
 [5480, 715, 0],
 [8000, 124, 0],
 [13243, 157, 0],
 [8059, 395, 0],
 [12434, 1024, 0],
 [8056, 336, 0],
 [6882, 919, 0],
 [1372, 915, 0],
 [6134, 886, 0],
 [4150, 1011, 0],
 [188, 736, 0],
 [11092, 75, 0],
 [3649, 600, 0],
 [5639, 459, 0],
 [9231, 529, 0]]

In [8]:
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

from recommenders.recommender import Recommender

from catboost import CatBoostRegressor

from collections import defaultdict

class ContentBasedUserItemRecommender(Recommender):
    """
    Linear recommender class based on user and item features.
    """
    
    def __init__(self, seed=6789, n_neg_per_pos=5, nn=False, cat=False, epochs=10):
        """
        Initialize base recommender params and variables.
        """
        self.model = LinearRegression()
        self.n_neg_per_pos = n_neg_per_pos
        
        self.recommender_df = pd.DataFrame(columns=['user_id', 'item_id', 'score'])
        self.users_df = None
        self.users_dict = None
        self.user_features = None
        self.user_features_dot_prod = None
        
        self.nn = nn
        self.cat = cat
        self.epochs = epochs
        
        self.uses_dot_product = True
        
        self.seed = seed
        self.rng = np.random.RandomState(seed=seed)
    
    def fit(self, interactions_df, users_df, items_df):
        """
        Training of the recommender.
        
        :param pd.DataFrame interactions_df: DataFrame with recorded interactions between users and items 
            defined by user_id, item_id and features of the interaction.
        :param pd.DataFrame users_df: DataFrame with users and their features defined by user_id and the user feature columns.
        :param pd.DataFrame items_df: DataFrame with items and their features defined by item_id and the item feature columns.
        """
        
        interactions_df = interactions_df.copy()
        
        user_ids = interactions_df['user_id'].unique().tolist()
        item_ids = interactions_df['item_id'].unique().tolist()
        
        # Prepare users_df and items_df
        
        users_df, user_features_dot_prod, user_features = prepare_users_df(interactions_df)
        
        self.users_df = users_df
        
        self.users_dict = users_df.copy().set_index('user_id').to_dict('index')
        
        self.user_features_dot_prod = user_features_dot_prod
        self.user_features = user_features
        
        items_df, item_features_dot_prod, item_features = prepare_items_df(interactions_df)   
        items_df = items_df.loc[:, ['item_id'] + item_features_dot_prod + item_features]
        
        # Generate negative interactions
        
        negative_interactions = generateNegativeInteractions(interactions_df, seed=self.seed, n_neg_per_pos=self.n_neg_per_pos)
        
        interactions_df = interactions_df.loc[:, ['user_id', 'item_id']]
        
        interactions_df.loc[:, 'interacted'] = 1

        interactions_df = pd.concat(
            [interactions_df, pd.DataFrame(negative_interactions, columns=['user_id', 'item_id', 'interacted'])])
        
        # Get the input data for the model
        
        interactions_df = pd.merge(interactions_df, users_df, on=['user_id'])
        interactions_df = pd.merge(interactions_df, items_df, on=['item_id'])
        
        if self.uses_dot_product:
            interactions_df[user_features_dot_prod] = interactions_df[user_features_dot_prod] \
                * interactions_df[item_features_dot_prod].values
            x = interactions_df.loc[:, user_features_dot_prod + user_features + item_features].values
        else:
            x = interactions_df.loc[:, user_features_dot_prod + user_features + item_features_dot_prod + item_features].values
        
        
        y = interactions_df['interacted'].values
        
        if self.cat:
            self.model.fit(x, y, silent=True, verbose=False)
        elif self.nn:
            self.model.fit(x, y, epochs=self.epochs, verbose=0)
        else:
            self.model.fit(x, y)
    
    def recommend(self, users_df, items_df, n_recommendations=1):
        """
        Serving of recommendations. Scores items in items_df for each user in users_df and returns 
        top n_recommendations for each user.
        
        :param pd.DataFrame users_df: DataFrame with users and their features for which recommendations should be generated.
        :param pd.DataFrame items_df: DataFrame with items and their features which should be scored.
        :param int n_recommendations: Number of recommendations to be returned for each user.
        :return: DataFrame with user_id, item_id and score as columns returning n_recommendations top recommendations 
            for each user.
        :rtype: pd.DataFrame
        """
        
        # Clean previous recommendations (iloc could be used alternatively)
        self.recommender_df = self.recommender_df[:0]
        
        item_df = items_df.copy()
        
        item_df, item_features_dot_prod, item_features = prepare_items_df(item_df)
        
        avg_user = self.users_df.copy().drop(columns='user_id').mean().to_frame().T
        
        
        # Score the items
    
        recommendations = pd.DataFrame(columns=['user_id', 'item_id', 'score'])
        
        for ix, user in users_df.iterrows():
            if user['user_id'] in self.users_dict:
                user_df = pd.DataFrame.from_dict({user['user_id']: self.users_dict[user['user_id']]}, orient='index')
            else:
                user_df = avg_user.copy()
            
            input_df = item_df.copy()
            
            # Calculate score
            
            input_df = input_df.merge(user_df, how='cross')
                
            if self.uses_dot_product:
                input_df[item_features_dot_prod] = input_df[item_features_dot_prod] \
                    * input_df[self.user_features_dot_prod].values
                if self.nn:
                    scores = self.model.predict(input_df.loc[:, item_features_dot_prod + self.user_features + item_features].values, verbose=0)
                    scores = scores.reshape(-1,)
                else:
                    scores = self.model.predict(input_df.loc[:, item_features_dot_prod + self.user_features + item_features].values)
            else:
                if self.nn:
                    scores = self.model.predict(input_df.loc[:, list(item_features_dot_prod) + list(item_features) + self.user_features_dot_prod + self.user_features].values, verbose=0)
                    scores = scores.reshape(-1,)
                else:   
                    scores = self.model.predict(input_df.loc[:, list(item_features_dot_prod) + list(item_features) + self.user_features_dot_prod + self.user_features].values)
            
            # Get highest scores
            
            chosen_ids = np.argsort(-scores)[:n_recommendations]
            
            # Recommendations
            
            recommendations = []
            for item_id in chosen_ids:
                recommendations.append(
                    {
                        'user_id': user['user_id'],
                        'item_id': item_id,
                        'score': scores[item_id]
                    }
                )
            
            user_recommendations = pd.DataFrame(recommendations)

            self.recommender_df = pd.concat([self.recommender_df, user_recommendations])

        return self.recommender_df
    
    
class LinearRegressionCBUIRecommender(ContentBasedUserItemRecommender):
    """
    Linear regression recommender class based on user and item features.
    """
    
    def __init__(self, seed=6789, n_neg_per_pos=5, **model_params):
        """
        Initialize base recommender params and variables.
        """
        super().__init__(seed=seed, n_neg_per_pos=n_neg_per_pos)
        self.model = LinearRegression()
        
        
class SVRCBUIRecommender(ContentBasedUserItemRecommender):
    """
    SVR recommender class based on user and item features.
    """
    
    def __init__(self, seed=6789, n_neg_per_pos=5, **model_params):
        """
        Initialize base recommender params and variables.
        """
        super().__init__(seed=seed, n_neg_per_pos=n_neg_per_pos)
        if 'kernel' in model_params:
            self.kernel = model_params['kernel']
        else:
            self.kernel = 'rbf'
        if 'C' in model_params:
            self.C = model_params['C']
        else:
            self.C = 1.0
        if 'epsilon' in model_params:
            self.epsilon = model_params['epsilon']
        else:
            self.epsilon = 0.1
        self.model = SVR(kernel=self.kernel, C=self.C, epsilon=self.epsilon)
        
    
class RandomForestCBUIRecommender(ContentBasedUserItemRecommender):
    """
    Random forest recommender class based on user and item features.
    """
    
    def __init__(self, seed=6789, n_neg_per_pos=5, **model_params):
        """
        Initialize base recommender params and variables.
        """
        super().__init__(seed=seed, n_neg_per_pos=n_neg_per_pos)
        if 'n_estimators' in model_params:
            self.n_estimators = int(model_params['n_estimators'])
        else:
            self.n_estimators = 100
        if 'max_depth' in model_params:
            self.max_depth = int(model_params['max_depth'])
        else:
            self.max_depth = 30
        if 'min_samples_split' in model_params:
            self.min_samples_split = int(model_params['min_samples_split'])
        else:
            self.min_samples_split = 30
        self.model = RandomForestRegressor(
            n_estimators=self.n_estimators, max_depth=self.max_depth, min_samples_split=self.min_samples_split, random_state=111)
    
    
class XGBoostCBUIRecommender(ContentBasedUserItemRecommender):
    """
    XGBoost recommender class based on user and item features.
    """
    
    def __init__(self, seed=6789, n_neg_per_pos=5, **model_params):
        """
        Initialize base recommender params and variables.
        """
        super().__init__(seed=seed, n_neg_per_pos=n_neg_per_pos)
        if 'n_estimators' in model_params:
            self.n_estimators = int(model_params['n_estimators'])
        else:
            self.n_estimators = 100
        if 'max_depth' in model_params:
            self.max_depth = int(model_params['max_depth'])
        else:
            self.max_depth = 30
        if 'min_samples_split' in model_params:
            self.min_samples_split = int(model_params['min_samples_split'])
        else:
            self.min_samples_split = 30
        if 'learning_rate' in model_params:
            self.learning_rate = model_params['learning_rate']
        else:
            self.learning_rate = 30
        self.model = GradientBoostingRegressor(
            n_estimators=self.n_estimators, max_depth=self.max_depth, min_samples_split=self.min_samples_split,
            learning_rate=self.learning_rate, random_state=seed)   
        
class NeuralNetworkCBUIRecommender(ContentBasedUserItemRecommender):
    """
    Neural Network recommender class based on user and item features.
    """
    
    def __init__(self, seed=6789, n_neg_per_pos=5, **model_params):
        
        if 'epochs' in model_params:
            self.epochs = model_params['epochs']
        else:
            self.epochs = 10
        
        super().__init__(seed=seed, n_neg_per_pos=n_neg_per_pos, nn=True, epochs=self.epochs)
        
        tf.random.set_seed(seed)
        tf.keras.utils.set_random_seed(seed)
        tf.config.experimental.enable_op_determinism()
            
        self.model = keras.Sequential(
            [
                keras.Input(shape=15),
                layers.Dense(32, activation='relu'),
                layers.Dropout(0.2),
                layers.Dense(32, activation='relu'),
                layers.Dropout(0.2),
                layers.Dense(16, activation='relu'),
                layers.Dense(1, activation='sigmoid')
            ]
        )
        
        self.model.compile(
            optimizer='adam',
            loss='mean_squared_error'
        )


class CatBoostRegressorCBUIRecommender(ContentBasedUserItemRecommender):
    """
    CatBoost recommender class based on user and item features.
    """
    
    def __init__(self, seed=6789, n_neg_per_pos=5, cat=True, **model_params):
        """
        Initialize base recommender params and variables.
        """
        super().__init__(seed=seed, n_neg_per_pos=n_neg_per_pos)
        if 'learning_rate' in model_params:
            self.learning_rate = model_params['learning_rate']
        else:
            self.learning_rate = 0.01
        if 'iterations' in model_params:
            self.iterations = model_params['iterations']
        else:
            self.iterations = 1000
        if 'l2_leaf_reg' in model_params:
            self.l2_leaf_reg = model_params['l2_leaf_reg']
        else:
            self.l2_leaf_reg = 1.0
        if 'depth' in model_params:
            self.depth = model_params['depth']
        else:
            self.depth = 5
            
        self.model = CatBoostRegressor(
            loss_function='RMSE',
            learning_rate=self.learning_rate,
            iterations=self.iterations,
            l2_leaf_reg=self.l2_leaf_reg,
            depth=self.depth,
            verbose=False
        )

# Quick test of the recommender

In [9]:
items_df = interactions_df.loc[:, ['item_id'] + base_item_features].drop_duplicates()

In [28]:
# Fit method
cb_user_item_recommender = RandomForestCBUIRecommender()
cb_user_item_recommender.fit(interactions_df, None, items_df)

In [29]:
# Recommender method

recommendations = cb_user_item_recommender.recommend(pd.DataFrame([[1], [2], [3], [4], [5]], columns=['user_id']), items_df, 10)

recommendations = pd.merge(recommendations, items_df, on='item_id', how='left')
display(recommendations)

,user_id,item_id,score,term,length_of_stay_bucket,rate_plan,room_segment,n_people_bucket,weekend_stay
0,1,23,0.966178,OffSeason,[4-7],Standard,[0-160],[2-2],True
1,1,56,0.964398,OffSeason,[2-3],Nonref,[0-160],[3-4],True
2,1,42,0.955583,OffSeason,[4-7],Standard,[160-260],[3-4],True
3,1,129,0.942980,OffSeason,[2-3],Nonref,[0-160],[3-4],False
4,1,60,0.937457,OffSeason,[2-3],Nonref,[0-160],[2-2],True
5,1,58,0.934320,OffSeason,[2-3],Nonref,[0-160],[2-2],False
6,1,36,0.927237,OffSeason,[4-7],Standard,[0-160],[3-4],True
7,1,33,0.923112,OffSeason,[2-3],Standard,[0-160],[2-2],True
8,1,506,0.918414,WinterVacation,[2-3],Standard,[0-160],[3-4],False
9,1,124,0.918414,WinterVacation,[2-3],Standard,[0-160],[2-2],False


# Tuning method

In [10]:
from evaluation_and_testing.testing import evaluate_train_test_split_implicit

seed = 6789

In [31]:
from hyperopt import hp, fmin, tpe, Trials, space_eval
import traceback

def tune_recommender(recommender_class, interactions_df, items_df, 
                     param_space, max_evals=1, show_progressbar=True, seed=6789):
    # Split into train_validation and test sets

    shuffle = np.arange(len(interactions_df))
    rng = np.random.RandomState(seed=seed)
    rng.shuffle(shuffle)
    shuffle = list(shuffle)

    train_test_split = 0.8
    split_index = int(len(interactions_df) * train_test_split)

    train_validation = interactions_df.iloc[shuffle[:split_index]]
    test = interactions_df.iloc[shuffle[split_index:]]

    # Tune

    def loss(tuned_params):
        recommender = recommender_class(seed=seed, **tuned_params)
        hr1, hr3, hr5, hr10, ndcg1, ndcg3, ndcg5, ndcg10 = evaluate_train_test_split_implicit(
            recommender, train_validation, items_df, seed=seed)
        print(f"@HR10 = {hr10}")
        return -hr10

    n_tries = 1
    succeded = False
    try_id = 0
    while not succeded and try_id < n_tries:
        try:
            trials = Trials()
            best_param_set = fmin(loss, space=param_space, algo=tpe.suggest, 
                                  max_evals=max_evals, show_progressbar=show_progressbar, trials=trials, verbose=True)
            display(best_param_set)
            succeded = True
        except:
            traceback.print_exc()
            try_id += 1
            
    if not succeded:
        return None
        
    # Validate
    
    params = space_eval(param_space, best_param_set)
    
    recommender = recommender_class(seed=seed, **params)

    results = [[recommender_class.__name__] + list(evaluate_train_test_split_implicit(
        recommender, {'train': train_validation, 'test': test}, items_df, seed=seed))]

    results = pd.DataFrame(results, 
                           columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

    display(results)
    
    return best_param_set

## Tuning of the recommender

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Tune your models using the code below. You only need to put the class name of your recommender and choose an appropriate parameter space.

In [32]:
param_space = {
    'n_neg_per_pos': hp.quniform('n_neg_per_pos', 1, 10, 1)
}

best_param_set = tune_recommender(LinearRegressionCBUIRecommender, interactions_df, items_df,
                                  param_space, max_evals=10, show_progressbar=True, seed=seed)

print("Best parameters:")
print(best_param_set)

@HR10 = 0.09013965298349556                           
@HR10 = 0.06220905628438426                                                       
@HR10 = 0.06136267456622937                                                       
@HR10 = 0.06093948370715192                                                       
@HR10 = 0.08971646212441811                                                       
@HR10 = 0.09013965298349556                                                       
@HR10 = 0.09013965298349556                                                       
@HR10 = 0.08929327126534067                                                       
@HR10 = 0.06178586542530681                                                       
@HR10 = 0.06093948370715192                                                       
100%|██████████| 10/10 [12:25<00:00, 74.53s/trial, best loss: -0.09013965298349556]


{'n_neg_per_pos': 8.0}

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,LinearRegressionCBUIRecommender,0.003055,0.012899,0.021724,0.071623,0.003055,0.008955,0.012622,0.02836


Best parameters:
{'n_neg_per_pos': 8.0}


# Final evaluation

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Run the final evaluation of your recommender and present its results against the Amazon recommender's results. You can present results for several of your recommenders. You just need to give the class name of your recommender and its tuned parameters below. If you present results for several recommenders, you should add a separate cell for each recommender and change the names of the DataFrames containing results.

In [11]:
from recommenders.amazon_recommender import AmazonRecommender

# LinearRegressionCBUIRecommender
linearRegressionCBUIRecommender = LinearRegressionCBUIRecommender(**{
    'n_neg_per_pos': 8.0
})

# XGBoostCBUIRecommender
xGBoostCBUIRecommender = XGBoostCBUIRecommender(**{
    'learning_rate': 0.013782403981999257, 
    'max_depth': 3.0, 
    'min_samples_split': 4.0, 
    'n_estimators': 197.0, 
    'n_neg_per_pos': 9.0
})

# CatBoostRegressorCBUIRecommender
catBoostRegressorCBUIRecommender = CatBoostRegressorCBUIRecommender(**{
    'iterations': 600, 
    'learning_rate': 0.02, 
    'depth': 6, 
    'l2_leaf_reg': 3.0,
    'n_neg_per_pos': 7.0
})

# AmazonRecommender
amazonRecommender = AmazonRecommender()

# List of recommenders
recommenders = [
    linearRegressionCBUIRecommender,
    xGBoostCBUIRecommender,
    catBoostRegressorCBUIRecommender,
    amazonRecommender
]

# Prepare list for all results
all_results = []

# Evaluate recommenders
for recommender in recommenders:
    result = [[type(recommender).__name__] + list(evaluate_train_test_split_implicit(recommender, interactions_df, items_df))]
    result = pd.DataFrame(result, columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])
    all_results.append(result)
    
# Concatenate all recommenders result into one DataFrame
all_results = pd.concat(all_results).reset_index(drop=True)

# Display evaluation results
display(all_results)

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,LinearRegressionCBUIRecommender,0.003055,0.012899,0.021724,0.071623,0.003055,0.008955,0.012622,0.028360
1,XGBoostCBUIRecommender,0.012559,0.025798,0.050577,0.102172,0.012559,0.020334,0.030664,0.047103
2,CatBoostRegressorCBUIRecommender,0.028174,0.058045,0.092668,0.202308,0.028174,0.045376,0.059127,0.094717
3,AmazonRecommender,0.035981,0.091650,0.132043,0.185336,0.035981,0.068037,0.084780,0.102033
